In [13]:
#pip install requests
#pip install python-dotenv
#pip install beautifulsoup4
#pip install openai

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import openai

# Cargar variables del archivo .env
load_dotenv()

# Configurar tu clave de API de OpenAI
openai.api_key = os.getenv("API_KEY")

# Descargar sitio url con parametro de etiqueta teatro
url = "https://www.rosario.gob.ar/inicio/agenda/buscar?etiquetas[333]=333"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

# Extraer texto relevante
contenedor = soup.select_one(".vista-agenda-filtros-avanzados > .govuk-grid-row")

if contenedor:
    agenda = contenedor.get_text(strip=True)
else:
    print("❌ No se encontró el contenedor. Verifica el selector CSS.")
    agenda = ""

In [14]:
conversation = [
  {
    "role": "system",
    "content": (
        "Actuás como un asistente experto en eventos culturales y recreativos de la ciudad de Rosario. "
    "Tu tarea es identificar los títulos de obras de teatro presentes en el texto fuente. "
    "Debés mostrar los títulos **exactamente como aparecen en el texto original**, sin omitir ninguna palabra, aunque incluyan expresiones como 'Funciones de', 'Obra:', 'Presentación de', etc. "
    "No debés resumir, reformular, corregir, traducir ni seleccionar partes del título: **copialo tal cual aparece**. "
    "El título debe ser tomado del bloque de texto que da a entender que se trata de una obra de teatro. "
    "Ignorá horarios, precios, lugares o descripciones, y no extraigas frases sueltas que no parezcan títulos. "
    "Si hay dudas, incluí toda la frase, sin modificarla."
    )
  },
  {
    "role": "user",
    "content": f"""Texto fuente con posibles obras de teatro:

{agenda}

IMPORTANTE:
- Si el texto está vacío o no contiene ninguna obra de teatro, respondé exclusivamente con: 'Por ahora no hay obras de teatro en la Agenda'.
- Si hay una o más obras, mostrá sus títulos exactamente como aparecen.
- Mostrá como máximo los primeros 5 títulos, sin completar ni inventar si hay menos.
- Comenzá la respuesta con: 'Estas son las obras de teatro que te sugiero:'."""
  },
  {
    "role": "user",
    "content": "¿Qué obras de teatro hay en Rosario?"
  }
]

In [15]:
response = openai.ChatCompletion.create(
    model="gpt-4o",  # este es el nombre correcto del modelo "04-mini"
    messages=conversation,
    temperature=0,
    max_tokens=150
)

print(response["choices"][0]["message"]["content"])

Estas son las obras de teatro que te sugiero:

1. La Noche de los Teatros Independientes de Rosario
2. La Noche de los Teatros Independientes en La Orilla Infinita
3. La Noche de los Teatros Independientes en Cultural de Abajo
4. La Noche de los Teatros Independientes en Club Fosse
5. La Noche de los Teatros Independientes en Tandava Sala


In [16]:
if response["choices"][0]["message"]["content"]:
    # Contenido completo generado por GPT, con los 5 títulos
    titulos = response["choices"][0]["message"]["content"]

    # Extraer el primer título de la lista
    primer_titulo = titulos.strip().split("\n")[0]
    # Limpiar número "1. Título"
    titulo_limpio = primer_titulo.strip().split(". ", 1)[-1]

    # Prompt para generar la imagen con el primer titulo de la lista
    prompt = f"""Crea una ilustración artística inspirada en la obra de teatro municipal correspondiente al título:
    {titulo_limpio}
    Tu tarea es representar visualmente el contenido que sugiere ese título: el tipo de evento, el ambiente, las personas, los objetos y escenarios que uno esperaría encontrar en una actividad cultural organizada por una ciudad.
    El estilo de la ilustración debe ser:
    - Colorido  
    - Realista  
    - Acogedor
    La escena debe estar ambientada en un centro cultural o teatro local, incluyendo detalles como:
    - Público diverso y participativo  
    - Escenografía o decoración del lugar  
    - Elementos culturales locales
    Si el título es demasiado abstracto y no se puede interpretar de forma clara, generá una escena alternativa que muestre una representación teatral genérica vista desde el público, con actores en el escenario y una ambientación cálida.
    Importante: transmití el espíritu comunitario del evento y la conexión con el entorno cultural urbano.
    """

    try:
        response = openai.Image.create(
            prompt=prompt,
            model="dall-e-3",
            n=1,
            size="1024x1024"
        )

        image_url = response['data'][0]['url']
        print("✅ Imagen generada:")
        print(image_url)

    except openai.error.OpenAIError as e:
        print("❌ Error al generar la imagen:", e)

    except Exception as e:
        print("❌ Otro error inesperado:", e)
else:
    print("❌ No se hay obras de teatro para generar imagenes.")

✅ Imagen generada:
https://oaidalleapiprodscus.blob.core.windows.net/private/org-GfpVRHFi9RRenarZhyfEvy9l/user-wZLS1o2mwY0IV0Lgbu12ArWb/img-xrZaQN0R9YqXEdItFbs69gHQ.png?st=2025-06-16T23%3A40%3A07Z&se=2025-06-17T01%3A40%3A07Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=cc612491-d948-4d2e-9821-2683df3719f5&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-06-16T12%3A40%3A07Z&ske=2025-06-17T12%3A40%3A07Z&sks=b&skv=2024-08-04&sig=XcIKPt/r9GEb3vpsKZZ205l4A0KSsXEKu10wAzV9x88%3D
